# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from sqlalchemy import create_eninge
engine = create_engine('postgresql://localhost:5432/programming_db')
connection = engine.connect()

from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base


ImportError: cannot import name 'create_eninge' from 'sqlalchemy' (C:\Users\Owner\anaconda\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\__init__.py)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,tasiilaq,100,GL,1666660548,95,65.6145,-37.6368,32.07,16.28
1,1,busselton,73,AU,1666660592,62,-33.6500,115.3333,61.34,10.54
2,2,azare,95,NG,1666660592,39,11.6765,10.1948,81.55,7.11
3,3,punta arenas,100,PH,1666660593,85,8.9897,125.3400,80.69,4.92
4,4,bredasdorp,100,ZA,1666660593,85,-34.5322,20.0403,60.82,5.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
# Using Lat and Lng as locations:
locations = cities_non_df[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = cities_non_df["Humidity"].astype(float)

gmaps.configure(api_key=g_key)
# Using Lat and Lng as locations:
locations = cities_non_df[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = cities_non_df["Humidity"].astype(float)

In [6]:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)

#plt.savefig("charts/humidity_heatmap.png") image saves with the image so doing a window snapshot
# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = cities_non_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
17,17,puerto ayora,0,ES,1666660597,57,36.5939,-6.2330,70.25,6.91
131,131,salalah,0,OM,1666660631,73,17.0151,54.0924,77.09,3.44
137,137,miandrivazo,0,MG,1666660633,68,-19.5167,45.4667,76.41,2.84
163,163,coconut creek,0,US,1666660641,78,26.3995,-81.8395,77.04,6.91
186,186,puerto escondido,0,ES,1666660648,57,36.5939,-6.2330,70.25,6.91
194,194,pozo colorado,0,CU,1666660651,81,22.4497,-80.1794,73.60,6.96
222,222,sur,0,OM,1666660660,38,22.5667,59.5289,79.47,5.26
229,229,surt,0,LY,1666660662,71,31.2089,16.5887,70.83,3.04
275,275,puerto leguizamo,0,ES,1666660648,57,36.5939,-6.2330,70.25,6.91
295,295,mar del plata,0,ES,1666660680,57,43.4002,-4.0094,70.02,9.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = filtered_weather_df.reset_index(drop=True)

hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "AcYSjRimwa_yFBHqohKRQJD4As69E-9ugw4RbR__eKeVRwZdS1AnLeidROImKltPr5ZUGpSTdd5fS7Rgh8wXpHu3uOxe4dBKeLasn23mFJV6zBm0A6fMjJa3STd3pvAJH9f0eyY2hUBOhNht09CuTy5KHIpx46OajrRMCBSMyQ9II8EiS6mxQFh6Q2UHKlselupslogpb_xiPcuu8OdpyBBdBugkVm0szsUtKwTvKgkIHPsxY0dgF_mBlkdQWvyzlWhsZQdWEu_B384KSumXyIDlcX2gOyA9StZmB6wXARDwP9iqdUSBOTkOe-9QAdMaK4NXY-yWMuFdxgfgONETbpODtG4YS7jfZ-vHiGjGR_TKwO2F77fzK_xsm4EaXFLdFjeDpOwspkl1xKP3a-MBKc8iPZKGVhnYy_BTiJv7EuHwRyErZiCwm8y48pHcLwYV4WFs",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 36.605051,
                    "lng": -6.220924999999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.60636287989271,
                        "lng": -6.219438320107277
                    },
                    "southwest": {
                        "lat": 36.6

{
    "html_attributions": [],
    "next_page_token": "AcYSjRjttPi6k1T5bpgP-Ba4Ok2CsPXuN3JCXKa1Cx-Tjibx4qu2LsuESo9MyZzFzXtgTrs977BRXUEYvKikLi38Hh-C4TCWZ_T3RoQtbtmyicEx8dz9hZcs79VLD6_to_H3dvUKAFGU2r_l_1n-cx6Jbkd2G0Evx_DN6cA7_38hpfbARdbK71EBtfoZHGMOVYyZqHKKs_jMAGgLp4CmW6XWZsZTA5FfryfCEjdv1xopDP_LiFWpo7Lv1GqtDlpzxPHAX-6Bkn8HKxK6fwUH8KFn1dlGmrohttdMTXsmC-XOZlmOGYh-7jnNMNkTAuuoDF4kiwYXHxm9hv5BZdcDUgTB7wjld-bklPcTzaUVZrGM6judfMkLYAhIT7OLsAmz1k17lF0nvhWVGIAywDfLcrDkJrD1xFZpujqeib3jQXcUJZ8tXhwOoSgzO_5r8dMTIiV7",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 17.031383,
                    "lng": 54.302784
                },
                "viewport": {
                    "northeast": {
                        "lat": 17.03295712989272,
                        "lng": 54.30409507989273
                    },
                    "southwest": {
                        "lat": 17.03025747010

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -19.5351949,
                    "lng": 45.4616398
                },
                "viewport": {
                    "northeast": {
                        "lat": -19.53384507010728,
                        "lng": 45.46298962989272
                    },
                    "southwest": {
                        "lat": -19.53654472989272,
                        "lng": 45.46028997010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "EDEN DE LA TSIRIBIHINA",
            "photos": [
                {
                    "height"

{
    "html_attributions": [],
    "next_page_token": "AcYSjRjHSJF9YHONidRnpv8LCwiZqD1f1UnED1txqjCETTalU6ztWpvjuGr4gUpPmchcgKsS-DBIJRi9u_TMGIoxDFf6ZjvYPKR99NXdK-v_2jsqs-QRX1juMe18QFJwvcCECP1cvV43NUJbaw6NQPokHkUrklb8pOL-_QPnF1B8fWBJNgSHhSqchoLTFbOCCICp7wI2Nn5md3Fpe2cBEtssBrQDKtmN08PgvnvM8QXpyPYEg8deBJXMiDFYVhIinxE7DcFnwpXVhBi-htYK25AFkng3z-wtPHmg0lVEk4pErCM-bIW6phgZhg3c0Va7Bz3z3mPns8ik_Ul8Lxxjo22fA8QHkkN4IqTjveYd3GDi3c-6oRsYQ6S4kpmaEhYLDoOikwMkdb21Vvd3N_9fZhWeWOxttKl9ZpRe2AznuXrfriwkTMRMVF1cxXEJZkF47XU2",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 26.6078176,
                    "lng": -81.6538061
                },
                "viewport": {
                    "northeast": {
                        "lat": 26.60899817989272,
                        "lng": -81.65242107010728
                    },
                    "southwest": {
                        "lat": 26.6062985

{
    "html_attributions": [],
    "next_page_token": "AcYSjRgp_u7MVLfmlt2BwIJEP4_D_yDHjXU0YK8LLHUkv0dc6m69FP39yrRV_0WI6DAxFH9iTbyni_LOwYfqaXqUUkOHMgOb0kHB2PlfYiVc74JaV18W0uHkjwO0TdEjs8qDyCoqhqBg0DL-4tkcbup6NoiA8tsL5VBCKa2CGnUlk2hyejym8wDrZciJZR9OxOa3rwMRJGxdZ1ehWYV9f43iySqgfGFb836OSac1TUThGR9KGt1mEV5LCkntRLKySL4nFkNku7ousX8R4CxtJgHzLKE9BcDEmARiP1ghuZevWW53R21EDT7iLw4UvRJ2EZfwEdsNJotp0aE6vv2Hd6xnPp7w1WQYg1VonX_IJVebPJXARxOqSJq3AhDlOY_r91ckr_Ma0earDBPKjZ17XinwjElXcNsZPGAtieYmmJEXMFs2Vczjh9Hi1LXiKUn9NR-g",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 36.605051,
                    "lng": -6.220924999999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.60636287989271,
                        "lng": -6.219438320107277
                    },
                    "southwest": {
                        "lat": 36.6

...

{
    "html_attributions": [],
    "next_page_token": "AcYSjRgL4AAbmeRyA7EXR1cbrWzi8s8aYTtXmAAvdbiLivVQ7o8-PIhbhFulvVZwNGsLTvFGrKU2PKAzy7UGUMKwUiehCWFrK8t75JZVBQNWDpCvAxX5LtRucD5fezebJRWACsIXVpkWp49xHuu9TxAMl5o3zmdPgEiNRiL5NpZYqWHRV8u0Mw2OzkCdKC95hfdRAd9BSDJx4tJS81-i2HgIY9ADBN2d8-9CjJhzldzZMb4eU5Sh8QjBnCXQuoUhjlLrWNdsVSOcH3n9bKxFVsP9YCiebHY_rH0hNkWfPXtDwIqWRlQq5_MvZWdZOG_0orvMr68yMAtNK0z1UMlrDR2OswCDsXaTGkVoSRoE2eAYF_ahTM_tw8SnlYcTJxtNnowcbUVACrlMD_H3ydXUoXF9aWEHcMz2zKEB2xxbVbcOiLxEAxdZtP6mub7fz7RLZ3pW",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 21.6314845,
                    "lng": 69.60907759999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 21.63286612989272,
                        "lng": 69.6104851798927
                    },
                    "southwest": {
                        "lat": 21.630

{
    "html_attributions": [],
    "next_page_token": "AcYSjRiYh0dh2G_xN3-03vQknhCCjmQAfHGFxFcrKSPnF97ooPRyk5FCJ8k9lsc8nPziswomBnui9nQnXqvFiGEyyEnVFKVnhVvk_U9qVqXtUk_pbOLYEzIVLbDXej9fHAsOBNOgTmwoqaWGDmRxyhhDfYUNQC7A1HJ10NLzZTX5vJLsE6Fkpkb7YzlF_YNZgibfmiiac4NlH5G6BFPHGnVW6pb8yvC0b-1LQnUS8Zmq2o93ZaRPMUKYpe4fC9mpXQCW-2kSBNIPhIZH6FO4lcU7mKSMuEY2ncZUZpHgvth4sQfg71nYF-fYRP-A6zrAPT_trKT1UM2U88vVOJIf4497Jg1QsKc2ZpQHH_YRRYyGJK-W5CdRQLHZEfb_HW76j26Ae-eNefkbBK29h7_KlHkdbJoCiykd-_SXalsjkHcKOj_SQ8Om3XWf-P91wxXe9wob",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.3802697,
                    "lng": 15.0918227
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.38159907989272,
                        "lng": 15.09319517989273
                    },
                    "southwest": {
                        "lat": 32.378899420

{
    "html_attributions": [],
    "next_page_token": "AcYSjRg8wQ2a7YuPb0AEMm2rNps6rD8oMlbz2tPwKssrWxaUGxCEnsxqkU7RHfwiNaKb0jmZPXc-cLITmq-4aO5OjX85vex6XWc-6OF7NsheZM2dxdWttCaZt4TBVXw8nDj-T8eP0hW4tbtalKr3onad-WPxEqG7r_9ZGeOVsOniSn_Y8ClbenoZyjUuyUpiWc-EV5FMQo9DdIlWlnjhDM4XqerU8B_URBD0f_xO3_OrHFGT6oWcjYkrBA2w3R1EGos6L_Z1J7kpvxMeLD9XXe7bj4_UgCrvhdx_Mr7b6sGRYbk5FNrmXIh55zYDdKUrofeDe7VpdhQa5nWc4koY3VlAh1P4ynu_AH1u12HO7Upfh5KbH8zEOkgdYRG_A35WpLG8ZJA9oFVq83u-Nn7BVHIObfS1WWF_FOIe_4qpIGd4jakR_SokDa-YJ_fXJyOu-vpO",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 38.5332889,
                    "lng": -123.277175
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.53479917989272,
                        "lng": -123.2758917701073
                    },
                    "southwest": {
                        "lat": 38.5320995

{
    "html_attributions": [],
    "next_page_token": "AcYSjRjbYYYOyqJIKqEgxF4zIhzKWksTCcVMBdSO6L0bzElQNzOvkmrThaiPohF3tN1-3cq8srLOA2uW2PN5zZoU3UA0SaxUCh2K3BANIHBok7ZbpjJFon7C4jmzoAfO9dgvBdQjv6po2RasirbgTAx8GneibHqGSYryDYeUrJ5-sKj1B6D9TFXMxjkOi1Z6CvVHmaRltrzO9bdUgEB5RgRwV7QP1NGunJ_BItTq8w2_wMRnUvXEYTKK15tgtRg8BhV7OdogBWD0zJEyabW4CzIgoBUXJg6b04muEF08XCXMa5qbpOko5SNc6-1DgGYHI0S6L7un3dX_2PzWZChUu-t_JCxbuXX5tO3Ev3oBaPJt-BeyQWjM31px7_YHBCCLnTNyC4FsctQ0s24TbyvRbv1P1qm93K3nhBrWndfcrNcsdZAlF744rLW86KNIlyQrx9yw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 19.4185956,
                    "lng": 72.8754676
                },
                "viewport": {
                    "northeast": {
                        "lat": 19.41990082989272,
                        "lng": 72.87693817989272
                    },
                    "southwest": {
                        "lat": 19.417201170

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,17,puerto ayora,0,ES,1666660597,57,36.5939,-6.2330,70.25,6.91,Duques de Medinaceli
1,131,salalah,0,OM,1666660631,73,17.0151,54.0924,77.09,3.44,Fanar Hotel and Residences
2,137,miandrivazo,0,MG,1666660633,68,-19.5167,45.4667,76.41,2.84,EDEN DE LA TSIRIBIHINA
3,163,coconut creek,0,US,1666660641,78,26.3995,-81.8395,77.04,6.91,Quality Inn & Suites Lehigh Acres Fort Myers
4,186,puerto escondido,0,ES,1666660648,57,36.5939,-6.2330,70.25,6.91,Duques de Medinaceli
5,194,pozo colorado,0,CU,1666660651,81,22.4497,-80.1794,73.60,6.96,Los Caneyes
6,222,sur,0,OM,1666660660,38,22.5667,59.5289,79.47,5.26,Ras Al Jinz Turtle Reserve
7,229,surt,0,LY,1666660662,71,31.2089,16.5887,70.83,3.04,عمارة السلطان للشقق المفروشة
8,275,puerto leguizamo,0,ES,1666660648,57,36.5939,-6.2330,70.25,6.91,Duques de Medinaceli
9,295,mar del plata,0,ES,1666660680,57,43.4002,-4.0094,70.02,9.22,Palacio de Arce


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
 
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))